In [ ]:
import pickle
import math
import pandas as pd
import numpy as np
import os
import pathlib
import tqdm
from functools import reduce
from scipy.stats import uniform, randint
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
import warnings
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

proj_dir = '/Users/nickbachelder/Desktop/Kaggle/Linemen'

os.chdir( os.path.join(proj_dir, 'code/clean') )

%run clean_objects.ipynb

os.chdir( os.path.join(proj_dir, 'code/modeling') )

%run metric_objects.ipynb
%run lstm_objects.ipynb

def get_player_names(nfl_ids, master_track):
    player_name_df = master_track.player_df.loc[:, ['nflId', 'displayName']].astype({'nflId': 'int32'})
    df = pd.DataFrame({'nflId' : nfl_ids}).astype({'nflId': 'int32'})
    df = df.copy().merge(player_name_df, on = 'nflId', how = 'left')
    return(df)

def download_metric_plot_df(master_track, play_id, model, folder):
    this = get_all_player_metrics_on_play(master_track, play_id, model)
    this = pd.DataFrame(this)
    this.columns = get_player_names(this.columns, master_track).displayName.values.tolist()
    this['time_after_snap'] = np.arange(0, this.shape[0]) * 0.1
    this.to_csv(f'{folder}/metric_plot_{play_id}.csv')

def download_play_prob_differences(master_track, play_id, model, folder):
    max_time_after = max(master_track.search_track_weeks(variables = ["playId"], variable_values = [play_id]).time_after_snap.values)
    max_frame = int(max_time_after * 10)
    original_prob = predict_play(master_track = master_track, play_id = play_id, model = model, normalize = True, replace_player = None)
    all_plays = master_track.play_details_df
    play_rushers = all_plays.loc[(all_plays.pff_role == 'Pass Rush') & (all_plays.playId == play_id)].nflId.tolist()
    probs_df = {str(player_id) : (np.array(predict_play(master_track = master_track, play_id = play_id, model = model, normalize = True, replace_player = player_id))).tolist()[0:(max_frame + 1)] for player_id in play_rushers}
    this = probs_df
    this = pd.DataFrame(this)
    original_prob = original_prob[:len(this)]
    this.columns = get_player_names(this.columns, master_track).displayName.values.tolist()
    this['time_after_snap'] = np.arange(0, this.shape[0]) * 0.1
    this['Original'] = original_prob
    describe = master_track.play_info_df.loc[master_track.play_info_df.playId == play_id, :].playDescription.values[0]
    this['describe'] = describe 

    this.to_csv(f'{folder}/prob_differences_{play_id}.csv')

def download_play_plot_df(master_track, play_id, model, folder):
    # get rush dat and play prob dat
    play_prob = predict_play(master_track = master_track, play_id = play_id, model = model, normalize = True)
    rusher_play_info = master_track.get_play(play_id).reset_index(drop = 1)
    rusher_play_info.time_after_snap = round(rusher_play_info.time_after_snap, 1)
    this = get_all_player_metrics_on_play(master_track = master_track, play_id = play_id, model = model)
    player_metrics = []
    for player_id in this.keys():
        metrics = this.get(player_id)
        metrics_df = pd.DataFrame({'nflId': float(player_id), 'rusher_metric' : metrics, 'time_after_snap' : np.array(range(0, len(metrics))) / 10})
        player_metrics.append(metrics_df)
    player_metrics_df = pd.concat(player_metrics)
    rusher_play_info =  rusher_play_info.merge(player_metrics_df, on = ['nflId', 'time_after_snap'], how = 'left')

    # get def 
    def_match = master_track.play_details_df.loc[(master_track.play_details_df.playId == play_id) & (master_track.play_details_df.pff_nflIdBlockedPlayer.notna()), ['playId', 'nflId', 'pff_nflIdBlockedPlayer']]
    defender_track = []
    for defender in def_match.nflId:
        this_def = master_track.search_track_weeks(variables = ["playId", "nflId"], variable_values = [play_id, defender])
        defender_track.append(this_def)
    defender_track = pd.concat(defender_track).merge(def_match, on = ['playId', 'nflId'], how = 'left').loc[:,['nflId', 'playId', 'time_after_snap', 'pff_nflIdBlockedPlayer', 'x', 'y']].rename(columns = {'pff_nflIdBlockedPlayer' : 'nflId', 'nflId' : 'blockerId', 'x' : 'x_block', 'y' : 'y_block'})

    # get qb track
    qb_track = master_track.get_qb_track_on_play(play_id).loc[:, ['playId', 'time_after_snap', 'x', 'y', 's', 'a']].rename(columns = {'x' : 'x_qb', 'y' : 'y_qb', 's' : 's_qb', 'a' : 'a_qb'})

    # merge def and qb dat
    defender_qb_track = defender_track.merge(qb_track, on = ['playId', 'time_after_snap'], how = 'left')
    defender_qb_track.time_after_snap = round(defender_qb_track.time_after_snap, 1)

    # prepare for merge
    rusher_play_info = rusher_play_info.loc[(rusher_play_info.time_after_snap >= 0) & (rusher_play_info.time_after_snap < len(play_prob) / 10), ['nflId', 'time_after_snap', 'rusher_metric', 'x', 'y']].copy().drop_duplicates()
    defender_qb_track = defender_qb_track.loc[(defender_qb_track.time_after_snap >= 0) & (defender_qb_track.time_after_snap < len(play_prob) / 10), ['blockerId', 'time_after_snap', 'x_block', 'y_block', 'x_qb', 'y_qb']].copy().drop_duplicates()

    # prob df

    times = [round(x, 1) for x in np.arange(0, np.max(rusher_play_info.time_after_snap.values) + 0.1, 0.1).tolist()]
    play_prob_cut = play_prob[:len(times)]
    prob_df = pd.DataFrame({'time_after_snap' : times, 'prob_disrupt' : play_prob_cut})

    # merge

    plot_dat = rusher_play_info.merge(defender_qb_track, on = ['time_after_snap'], how = 'left')
    plot_dat = plot_dat.copy().merge(prob_df, on = ['time_after_snap'], how = 'left')

    # add play descript 

    describe = master_track.play_info_df.loc[master_track.play_info_df.playId == play_id, :].playDescription.values[0]
    plot_dat['describe'] = describe 

    # add names

    plot_dat['displayName'] = plot_dat.apply(lambda x: get_player_names([x['nflId']], master_track).displayName.values[0], axis=1)

    plot_dat.to_csv(f'{folder}/play_plot_{play_id}.csv')